# this is used to filter the cell images which are segmented incompletely

In [ ]:
cell_category = 'type4' # user's input, can choose type1, 2, 3, and 4
cell_dict = {'type1': 'BJ Primary', 'type2': 'BJ hTERT', 'type3': 'BJ SV40T', 'type4': 'BJ Ras'}
threshold_edge_1 = 0.12  # user's input, smaller -> more strict, same as threshold_edge_2
threshold_size_1 = 75 * 75 # user's input, height * width, bigger -> more strict, same as threshold_size_2
threshold_edge_2 = 0.16
threshold_size_2 = 100 * 100

In [ ]:
import tifffile
import os
import numpy as np

input_dir = 'C:\\Users\\15967\Desktop\\ameer-s-work-main' + '\\' + cell_category # input image path
output_dir = 'D:\\cell_formal\\OneDrive_2025-06-27\\Oscar_images\\' + cell_dict[cell_category] + '\\' + 'cell_segmented' # output image path
os.makedirs(output_dir, exist_ok=True)
output_good_dir = os.path.join(output_dir, 'good')
os.makedirs(output_good_dir, exist_ok=True)
output_bad_dir = os.path.join(output_dir, 'bad')
os.makedirs(output_bad_dir, exist_ok=True)

In [ ]:
def surrounding_pixel_percent(image):
    total_num = 2 * image.shape[0] + 2 * image.shape[1] - 4 # total number of pixels in the surrounding
    nonzero_pixel = 0.0
    for i in range(image.shape[0]):
        if np.mean(image[i,0,:]) > 0:
            nonzero_pixel += 1
        if np.mean(image[i,-1,:]) > 0:
            nonzero_pixel += 1
    for i in range(1, image.shape[1] - 1):
        if np.mean(image[0,i,:]) > 0:
            nonzero_pixel += 1
        if np.mean(image[-1,i,:]) > 0:
            nonzero_pixel += 1
    return float(nonzero_pixel / total_num)

In [ ]:
for subdir, _, files in os.walk(input_dir):
    print(subdir)
    print(files)
    subdir_name = subdir.split('\\')[-1]
    for file in files:
        if file.lower().endswith(('.tif', '.tiff')):
            file_path = os.path.join(subdir, file)
            img = tifffile.imread(file_path) # read the segmented cell images
            # use size to filter
            if img.shape[0] * img.shape[1] <= threshold_size_1:
                output_path = os.path.join(output_bad_dir, cell_category + '_' + subdir_name + '_' + file)
                tifffile.imwrite(output_path, img)
            elif threshold_size_1 < img.shape[0] * img.shape[1] <= threshold_size_2:
                # use surrounding pixels to filter, img should be in ndarray format
                assert type(img) == np.ndarray
                if surrounding_pixel_percent(img) > threshold_edge_1:
                    output_path = os.path.join(output_bad_dir, cell_category + '_' + subdir_name + '_' + file)
                    tifffile.imwrite(output_path, img)
                else:
                    output_path = os.path.join(output_good_dir, cell_category + '_' + subdir_name + '_' + file)
                    tifffile.imwrite(output_path, img)
            else:
                assert type(img) == np.ndarray
                if surrounding_pixel_percent(img) > threshold_edge_2:
                    output_path = os.path.join(output_bad_dir, cell_category + '_' + subdir_name + '_' + file)
                    tifffile.imwrite(output_path, img)
                else:
                    output_path = os.path.join(output_good_dir, cell_category + '_' + subdir_name + '_' + file)
                    tifffile.imwrite(output_path, img)